In [ ]:
# Importing libraries
import pandas as pd
def recommendation():
    
    # Read csv file into a pandas dataframe
    df = pd.read_csv("books.csv")
    df = df[df.language_code =='eng']
    df['Title'] = df['title'].str.split('(').str[0]
    print("Enter rating w/wo <rating> or random for book suggestion")
    search= input("enter to search ")
    if search=="random":
        df_new= df.sample(replace=True)[['Title', 'authors',"rating"]]
        print("Title: ", df_new[['Title']].to_string(index=False, header=False))
        print("Author: ", df_new[['authors']].to_string(index=False, header=False))
        print("Rating: ", df_new[['rating']].to_string(index=False, header=False))
        return "Enjoy!"
    elif "rating" in search:
            search=search.split()
            rate=float(search[1])
            df_rate = df[df['rating'].astype(float) >= rate]
            df_new= df_rate.sample(replace=True)[['Title', 'authors',"rating"]]
            print("Title: ", df_new[['Title']].to_string(index=False, header=False))
            print("Author: ", df_new[['authors']].to_string(index=False, header=False))
            print("Rating: ", df_new[['rating']].to_string(index=False, header=False))
            return "Enjoy!"
print(recommendation())

In [42]:
def author_search():
    # Read csv file into a pandas dataframe
    df = pd.read_csv("books.csv")
    df = df[df.language_code =='eng']

    #reading and cleaning author's name
    auth_inp = input("Enter the author's name: ")
    clean_auth_inp=auth_inp.lstrip().rstrip().title()
    
    #showing title, rating and author for selected name
    author_df = df[df.authors.str.find(clean_auth_inp)>=0][['title','authors','rating']]
    print(author_df)
    return None
author_search()


Enter the author's name:     J.K. ROWlinG
                                                   title  \
0      Harry Potter and the Half-Blood Prince (Harry ...   
1      Harry Potter and the Order of the Phoenix (Har...   
2      Harry Potter and the Chamber of Secrets (Harry...   
3      Harry Potter and the Prisoner of Azkaban (Harr...   
4      Harry Potter Boxed Set  Books 1-5 (Harry Potte...   
6           Harry Potter Collection (Harry Potter  #1-6)   
613    Harry Potter Schoolbooks Box Set: Two Classic ...   
615    Harry Potter and the Half-Blood Prince (Harry ...   
1233   Harry Potter and the Prisoner of Azkaban (Harr...   
4416   Harry Potter and the Chamber of Secrets (Harry...   
8876   Harry Potter and the Sorcerer's Stone (Harry P...   
10344  Fantastic Beasts and Where to Find Them (Hogwa...   
10679  Harry Potter and the Goblet of Fire (Harry Pot...   

                           authors rating  
0       J.K. Rowling/Mary GrandPré   4.57  
1       J.K. Rowling/Mary Gra